## Main toolbox

In [9]:
""" Import the libraries required by the program"""
#------------------------------------------------------------------------------------------------------

# GEE Python API
import ee

# geemap:A Python package for interactive mapping with Google Earth Engine, ipyleaflet, and ipywidgets
# Documentation: https://geemap.org
import geemap
from geemap import ee_basemaps

# geetols: Google earth engine tools
# https://github.com/gee-community/gee_tools
import geetools
from geetools import tools

# hydrafloods: Hydrologic Remote Sensing Analysis for Floods
#https://github.com/Servir-Mekong/hydra-floods
import hydrafloods as hf
from hydrafloods import geeutils

# Ipywidgets for GUI design
import ipywidgets as ipw
from IPython.display import display
from ipywidgets import HBox, VBox, Layout

# A simple Python file chooser widget for use in Jupyter/IPython in conjunction with ipywidgets
# https://pypi.org/project/ipyfilechooser/
from ipyfilechooser import FileChooser

# Plotly Python API for interactive graphing
import plotly
import plotly.express as px
import plotly.graph_objs as go

# Pandas -  Python Data Analysis Library for data analysis and manipulation
import pandas as pd

# Miscellaneous Python modules
from datetime import datetime, timedelta
import os

#----------------------------------------------------------------------------------------------------



""" UI Design"""
#---------------------------------------------------------------------------------------------------
# Program Title
Title_text =  ipw.HTML(
    "<h3 class= 'text-center'><font color = 'blue'>Python-GEE Surface Water Analyzer Toolbox v.1.0.0</font>")
style = {'description_width': 'initial'}


# Image Processing Tab
#************************************************************************************************
# Image Parameters UI
dataset_description = ipw.Label('Satellite Imagery Parameters')
dataset_Label = ipw.Label('Select Dataset:', layout=Layout(margin='5px 0 0 5px')) #top right bottom left
DatasetType_options = ['Landsat', 'Sentinel-1', 'Sentinel-2', 'USDA NAIP' ]
DatasetType_dropdown = ipw.Dropdown(options = DatasetType_options, value = None,
                                   layout=Layout(width='150px', margin='5px 0 0 5px'))
datasetType = HBox([dataset_Label, DatasetType_dropdown])

# Study period definition
#************************************************************************************************
# Start date picker
lbl_start_date = ipw.Label('Start Date:', layout=Layout(margin='5px 0 0 5px'))
start_date = ipw.DatePicker(value = datetime.now()-timedelta(7), disabled=False, 
                            layout=Layout(width='150px', margin='5px 0 0 30px'))
start_date_box = HBox([lbl_start_date, start_date])

# End date picker
lbl_end_date = ipw.Label('End Date:', layout=Layout(margin='5px 0 0 5px'))
end_date = ipw.DatePicker(value = datetime.now(), disabled=False, 
                          layout=Layout(width='150px', margin='5px 0 0 34px'))
end_date_box = HBox([lbl_end_date, end_date])

datePickers = VBox([start_date_box, end_date_box])


# Cloud threshold for filtering data
#************************************************************************************************
# Set cloud threshold
cloud_threshold = ipw.IntSlider(description = 'Cloud Threshold:',
                                orientation = 'horizontal',
                                 value = 50,
                                 step = 5,
                               style = style)

imageParameters = VBox([dataset_description, datasetType, datePickers, cloud_threshold], 
                   layout=Layout(width='305px', border='solid 2px black'))


# Study Area definition
#************************************************************************************************
# Option to use a map drawn boundary or upload shapefile
StudyArea_description = ipw.Label('Study Area Definition')
user_preference = ipw.RadioButtons(options=['Upload shapefile', 'Map drawn boundary'], value='Upload shapefile')
file_selector = FileChooser(description = 'Upload', filter_pattern = "*.shp", use_dir_icons = True)

# Retrieve and process satellite images
#***********************************************************************************************
# Button to retrieve and process satellite images from the GEE platform
imageProcessing_Button = ipw.Button(description = 'Process images',
                                    tooltip='Click to process images', button_style = 'info',
                                   layout=Layout(width='150px', margin='5px 0 0 50px'))

# Study area UI and process button container
# ************************************************************************************************
StudyArea = VBox(children = [StudyArea_description, user_preference, file_selector, imageProcessing_Button], 
                   layout=Layout(width='300px', border='solid 2px black', margin='0 0 0 10px'))



# Results UI for displaying number and list of files
#*****************************************************************************************************
lbl_results = ipw.Label('Processing Results')
lbl_images = ipw.Label('No. of processed images:')
lbl_RetrievedImages = ipw.Label()
display_no_images = HBox([lbl_images, lbl_RetrievedImages])
lbl_files = ipw.Label('List of files:')
lst_files = ipw.Select(layout=Layout(width='360px', height='100px'))

image_Results = VBox([lbl_results, display_no_images, lbl_files, lst_files], 
                   layout=Layout(width='400px', border='solid 2px black', margin='0 0 0 10px'))


# Container for Image Processing Tab
#************************************************************************************************
imageProcessing_tab = HBox([imageParameters, StudyArea, image_Results])


# Water Extraction Tab
#*************************************************************************************************
# Water extraction indices
water_index_options = ['NDWI','MNDWI','DSWE', 'AWEInsh', 'AWEIsh']
lbl_indices = ipw.Label('Water Index:', layout=Layout(margin='5px 0 0 5px'))
water_indices = ipw.Dropdown(options = water_index_options,
                             value = 'NDWI',
                            layout=Layout(width='100px', margin='5px 0 0 26px'))
display_indices = HBox([lbl_indices, water_indices])

# Color widget for representing water
lbl_color = ipw.Label('Color:', layout=Layout(margin='5px 0 0 5px'))
index_color =  ipw.ColorPicker(concise = False,
                              value = 'blue',
                              layout=Layout(width='100px', margin='5px 0 0 64px'))
display_color_widget = HBox([lbl_color, index_color])

# Water index threshold selection
lbl_threshold = ipw.Label('Index threshold:', layout=Layout(margin='5px 0 5px 5px'))
index_threshold = ipw.BoundedFloatText(value=0.0, min = -1.0, max = 1.0, step = 0.1,
                                       layout=Layout(width='100px', margin='5px 0 0 5px'))

display_threshold_widget = HBox([lbl_threshold, index_threshold])

water_index_Box = VBox([display_indices, display_color_widget, display_threshold_widget],
                      layout=Layout(width='220px', border='solid 2px black'))

extractWater_Button = ipw.Button(description = 'Extract Water',
                                tooltip='Click to extract surface water', 
                                button_style = 'info',
                                layout=Layout(width='150px', margin='5px 0 0 20px'))

Extraction_tab = HBox([water_index_Box, extractWater_Button])


# Spatial Analysis Tab
#**************************************************************************************************

water_Frequency_button = ipw.Button(description = 'Compute Water Frequency',
                                    tooltip='Click to compute water occurence frequency',
                                    button_style = 'info', layout=Layout(width='200px'))

sharpenImage_Button = ipw.Button(description = 'Pan Sharpening',
                                tooltip='Click to sharpen image', button_style = 'info',
                                layout=Layout(width='200px'))

removeCloud_Button = ipw.Button(description = 'Remove Clouds',
                                tooltip='Click to remove clouds', button_style = 'info',
                               layout=Layout(width='200px'))

Spatial_Analysis_Tab = VBox([water_Frequency_button, sharpenImage_Button, removeCloud_Button])


# Ploting and Statistics Tab
#***************************************************************************************************
area_unit = ipw.Dropdown(options = ['Square Km', 'Hectares', 'Acre'],
                             value = 'Square Km',
                            description = 'Unit for water surface area:',
                            style=style,
                            tooltip='Select unit for areas')

plot_button = ipw.Button(description = 'Plot',
                                    tooltip='Click to plot graph', button_style = 'info')
plotting_box = VBox([area_unit, plot_button], layout=Layout(width='310px', border='solid 2px black'))

folder_selector1 = FileChooser(description = 'Select Folder', show_only_dirs = True, use_dir_icons = True)
folder_selector1.title = '<b>Select a folder</b>'
folder_selector1.default_path = os.getcwd()

saveFile_name = ipw.Text(description='File Name:', layout=Layout(width='200px'))
save_data_button = ipw.Button(description = 'Save Data',
                                    tooltip='Click to save computed areas to file', button_style = 'info')


save_box = VBox(children=[folder_selector1, saveFile_name, save_data_button],
               layout=Layout(width='310px', border='solid 2px black', margin='0 0 0 10px'))
plot_stats_tab = HBox(children=[plotting_box, save_box])

# Downloads Tab
#***************************************************************************************************
files_to_download = ipw.RadioButtons(options=['Satellite Images', 'Water Mask', 'Water Frequency'], 
                                     value='Satellite Images', description='Files to download:', style = style)

download_location = ipw.RadioButtons(options=['Google Drive', 'Local Disk'], 
                                     value='Google Drive', description='Download Location:', style = style)

folder_name = ipw.Text(description='Folder Name:')

folder_selector = FileChooser(description = 'Select Folder', show_only_dirs = True, use_dir_icons = True)
folder_selector.title = '<b>Select a folder</b>'
folder_selector.default_path = os.getcwd()

download_button = ipw.Button(description = 'Download',
                                    tooltip='Click to plot download water images', button_style = 'info')


download_settings = VBox(children=[files_to_download, download_location, folder_name])

download_tab = HBox([download_settings, download_button])

# Functions to control UI changes and parameter settings
#****************************************************************************************************

def datatype_index_change(change):
    """
    Function to set image visualization parameters, hide or show some UI componets and
    show water indices that are applicable to the type of satellite image selected
        
    args:
        None

    returns:
        None
    """
    try:
        global img_type
        global visParams
        global cloud_metadata
        global water_index_options
        if DatasetType_dropdown.value == 'Landsat':
            visParams = {'bands': ['red', 'green', 'blue'],
                  'min': 0,
                  'max': 3000,
                  'gamma': [0.95, 1.1, 1]}
            cloud_threshold.disabled = False
            water_indices.disabled = False
            index_color.disabled = False
            cloud_metadata = 'CLOUD_COVER'
            index_threshold.disabled = False
            water_indices.options = ['NDWI','MNDWI','DSWE','AWEInsh', 'AWEIsh']
        elif DatasetType_dropdown.value == 'Sentinel-1':
            visParams = {'min': -25,'max': 5}
            cloud_threshold.disabled = True
            water_indices.disabled = True
            index_color.disabled = True
            index_threshold.disabled = True
        elif DatasetType_dropdown.value == 'Sentinel-2':
            visParams = {'bands': ['red', 'green', 'blue'],
              'min': 0.0,
              'max': 3000}
            cloud_threshold.disabled = False
            water_indices.disabled = False
            index_color.disabled = False
            cloud_metadata = 'CLOUDY_PIXEL_PERCENTAGE'
            index_threshold.disabled = False
            water_indices.options = ['NDWI','MNDWI','AWEInsh', 'AWEIsh']
        elif DatasetType_dropdown.value == 'USDA NAIP':
            visParams = {'bands': ['R', 'G','B'],
                        'min': 0.0,
                        'max': 255.0}
            index_threshold.disabled = False
            water_indices.disabled = False
            index_color.disabled = False
            water_indices.options = ['NDWI']
            
    except Exception as e:
             print(e)

DatasetType_dropdown.observe(datatype_index_change, 'value')


def showFileSelector(button):
    """
    Function to show or hide shapefile upload widget
        
    args:
        None

    returns:
        None
    """
    if button['new']:
        StudyArea.children = [StudyArea_description, user_preference,imageProcessing_Button]
    else:
        StudyArea.children = [StudyArea_description, user_preference, file_selector, imageProcessing_Button]

# Link widget to file selector function
user_preference.observe(showFileSelector, names='index')


def showLocationSelector(button):
    """
    Function to show or hide folder selector
        
    args:
        None

    returns:
        None
    """
    if button['new']:
        download_settings.children = [files_to_download, download_location, folder_selector]
    else:
        download_settings.children = [files_to_download, download_location, folder_name]
        
# Link widget to folder selector function
download_location.observe(showLocationSelector, names='index')

# Full UI
#***************************************************************************************************
tab_children = [imageProcessing_tab, Extraction_tab, Spatial_Analysis_Tab, plot_stats_tab, download_tab]

tab = ipw.Tab()
tab.children = tab_children

# changing the title of the first and second window
tab.set_title(0, 'Image Processing')
tab.set_title(1, 'Water Extraction')
tab.set_title(2, 'Spatial Analysis')
tab.set_title(3, 'Plotting & Stats')
tab.set_title(4, 'Download Images')

GUI = VBox([Title_text,tab])

# Plotting outputs and feedback to user
#***************************************************************************************************
plot_output = ipw.Output()
feedback = ipw.Output()
OUTPUTS = VBox([plot_output, feedback])

# create map instance
Map =  geemap.Map()
old_basemap = Map.layers[-1] 
Map.substitute_layer(old_basemap, ee_basemaps['HYBRID'])


def reset_map(self):
    Map.clear_layers()
    Map.add_basemap(basemap="ROADMAP")

def load_shapefile():
    """
    Function to laod shapefile
        
    args:
        None

    returns:
        ee user boundary
    """
    file  = file_selector.selected   
    user_boundary = geemap.shp_to_ee(file)
    Map.addLayer(user_boundary, {}, 'User boundary')
    Map.center_object(user_boundary, 15)
    return user_boundary

def maskS2clouds(image):
    """
    Function to mask out clouds from Sentinel-2 images
        
    args:
        Sentinel-2 image

    returns:
        Cloud masked image
    """
    qa = image.select('pixel_qa')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
    .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return (image.updateMask(mask).copyProperties(image, ['system:time_start']))

def maskLandsatclouds(image):
    """
    Function to mask out clouds from Landsat images
        
    args:
        Landsat image

    returns:
        Cloud masked image
    """
    qa = image.select('pixel_qa')
    #cloudsShadowBitMask = 1 << 7
    cloudsBitMask = 1 << 4
    mask = qa.bitwiseAnd(cloudsBitMask).eq(0) # \
    #.And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return (image.updateMask(mask).copyProperties(image, ['system:time_start']))  
    
def process_images(self):
    """
    Function to retrieve and process satellite images from GEE platform
        
    args:
        None

    returns:
        None
    """
    with plot_output:
        plot_output.clear_output()
        try:
            global filtered_Collection
            global filtered_landsat
            global clipped_images
            global imageType
            global dates
            global site
            global img_scale
            global file_list
            global StartDate
            global EndDate


            # Reset map
            #Map.clear_layers()
            #Map.add_basemap(basemap = "ROADMAP")

            lbl_RetrievedImages.value = 'Processing....'
            cloud_thresh = cloud_threshold.value

            # Define study area based on user preference
            if user_preference.index == 0:
                site = load_shapefile()
            else:
                site = ee.FeatureCollection(Map.draw_last_feature)

            # get widget values
            imageType = DatasetType_dropdown.value
            StartDate = ee.Date.fromYMD(start_date.value.year,start_date.value.month,start_date.value.day)
            EndDate = ee.Date.fromYMD(end_date.value.year,end_date.value.month,end_date.value.day)

            # Function to clip images
            def clipImages(img):
                """
                Function to clip images

                args:
                    Image

                returns:
                    Clipped image
                """
                clipped_image = img.clip(site).copyProperties(img, ['system:time_start', 'system:id'])
                return clipped_image

            def load_Sentinel1():
                """
                Function to retrieve and filter Sentinel-1 images

                args:
                    None

                returns:
                    Image collection of Sentinel-1 images
                """
                filtered_col = ee.ImageCollection('COPERNICUS/S1_GRD')\
                    .filterDate(StartDate,EndDate)\
                    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
                    .filterMetadata('transmitterReceiverPolarisation', 'equals',['VV','VH'])\
                    .filterMetadata('resolution_meters', 'equals', 10)\
                    .filterBounds(site)\
                    .sort('system:time_start')
                return filtered_col

            def load_Sentinel2():
                """
                Function to retrieve and filter Sentinel-2 images

                args:
                    None

                returns:
                    Image collection of Sentinel-2 images
                """
                filtered_col = ee.ImageCollection('COPERNICUS/S2')\
                    .filterDate(StartDate,EndDate)\
                    .filterBounds(site)\
                    .filter(ee.Filter.lt(cloud_metadata, cloud_thresh))\
                    .sort('system:time_start')\
                    .select(['B2','B3','B4','B8','B11','B12','QA60'], ['blue','green','red','nir','swir1','swir2','pixel_qa'])
                return filtered_col

            def load_Landsat():
                """
                Function to retrieve and filter Landsat images

                args:
                    None

                returns:
                    Image collection of Landsat images
                """
                # Define Landsat surface reflectance bands
                sensor_band_dict = ee.Dictionary({
                                    'l8' : ee.List([1,2,3,4,5,6,10]),
                                    'l7' : ee.List([0,1,2,3,4,6,9]),
                                    'l5' : ee.List([0,1,2,3,4,6,9]),
                                    'l4' : ee.List([0,1,2,3,4,6,9])
                                    })
                # Sensor band names corresponding to selected band numbers
                bandNames = ee.List(['blue','green','red','nir','swir1','swir2','pixel_qa'])
                # ------------------------------------------------------
                # Landsat 4 - Data availability Aug 22, 1982 - Dec 14, 1993
                ls4 = ee.ImageCollection('LANDSAT/LT04/C01/T1_SR') \
                          .filterBounds(site.geometry()) \
                          .select(sensor_band_dict.get('l4'), bandNames)

                # Landsat 5 - Data availability Jan 1, 1984 - May 5, 2012
                ls5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
                          .filterBounds(site.geometry()) \
                          .select(sensor_band_dict.get('l5'), bandNames)

                # Landsat 7 - Data availability Jan 1, 1999 - Aug 9, 2016
                # SLC-off after 31 May 2003
                ls7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
                              .filterDate('1999-01-01', '2003-05-31') \
                              .filterBounds(site.geometry()) \
                              .select(sensor_band_dict.get('l7'), bandNames)

                # Post SLC-off; fill the LS 5 gap
                # -------------------------------------------------------
                # Landsat 7 - Data availability Jan 1, 1999 - Aug 9, 2016
                # SLC-off after 31 May 2003
                ls7_2 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
                              .filterDate('2012-05-05', '2014-04-11') \
                              .filterBounds(site.geometry()) \
                              .select(sensor_band_dict.get('l7'), bandNames)

                # --------------------------------------------------------
                # Landsat 8 - Data availability Apr 11, 2014 - present
                ls8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
                              .filterBounds(site.geometry()) \
                              .select(sensor_band_dict.get('l8'), bandNames)

                # Merge landsat collections
                l4578 = ee.ImageCollection(ls4 \
                          .merge(ls5) \
                          .merge(ls7) \
                          .merge(ls7_2) \
                          .merge(ls8).sort('system:time_start')) \
                          .filterDate(StartDate, EndDate)\
                        .filter(ee.Filter.lt('CLOUD_COVER', cloud_thresh))

                return l4578

            def load_NAIP():
                """
                Function to retrieve and filter NAIP images

                args:
                    None

                returns:
                    Image collection of NAIP images
                """
                filtered_col = ee.ImageCollection('USDA/NAIP/DOQQ')\
                    .filterDate(StartDate,EndDate)\
                    .filterBounds(site)\
                    .sort('system:time_start')
                return filtered_col

            def smooth(img):
                """
                Function to smoothen Sentinel-1 images based on focal median

                args:
                    None

                returns:
                    Image collection of NAIP images
                """
                #smoothed_image = img.focal_median(float('50'),'circle', 'meters')
                return img.addBands(img.select('VV').focal_median(float('50'),'circle', 'meters'))#.rename('VV_smoothed'))

            # filter image collection based on date, study area and cloud threshold(depends of datatype)
            if imageType == 'Landsat':
                filtered_landsat = load_Landsat()
                filtered_Collection = filtered_landsat.map(maskLandsatclouds)
            elif imageType == 'Sentinel-2':
                Collection_before = load_Sentinel2()
                filtered_Collection = Collection_before.map(maskS2clouds)
            elif imageType == 'Sentinel-1':
                Collection_before = load_Sentinel1()
                # apply speckle filter algorithm or smoothing
                filtered_Collection = Collection_before.map(hf.lee_sigma)
                #filtered_Collection = Collection_before.map(smooth)
            elif imageType == 'USDA NAIP':
                filtered_Collection = load_NAIP()

            # Clip images to study area
            clipped_images = filtered_Collection.map(clipImages)
            # Mosaic same day images
            clipped_images = tools.imagecollection.mosaicSameDay(clipped_images)


            # Add images to map
            # Add first image in collection to Map
            first_image = clipped_images.first()
            if imageType == 'Sentinel-1':
                img_scale = 10
            else:
                bandNames = first_image.bandNames().getInfo()
                img_scale = first_image.select(str(bandNames[0])).projection().nominalScale().getInfo()

            Map.addLayer(clipped_images, visParams, imageType)

            # Get no. of processed images
            no_of_images = filtered_Collection.size().getInfo()

            # Display number of images
            lbl_RetrievedImages.value = str(no_of_images)

            # List of files
            file_list = filtered_Collection.aggregate_array('system:id').getInfo()
            # display list of files
            lst_files.options = file_list

        except Exception as e:
                 print(e)
                 print('An error occurred during processing.')
                    


# Return the DN that maximizes interclass variance in the region        
def otsu(histogram):
    """
    Function to use Otsu algorithm to compute DN that maximizes interclass variance in the region 

    args:
        Histogram

    returns:
        Otsu's threshold
    """
    counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
    size = means.length().get([0])
    total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
    sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
    mean = sum.divide(total)
    indices = ee.List.sequence(1, size)
    
    # Compute between sum of squares, where each mean partitions the data.
    def func_bss(i):
        aCounts = counts.slice(0, 0, i)
        aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
        aMeans = means.slice(0, 0, i)
        aMean = aMeans.multiply(aCounts) \
            .reduce(ee.Reducer.sum(), [0]).get([0]) \
            .divide(aCount)
        bCount = total.subtract(aCount)
        bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
        return aCount.multiply(aMean.subtract(mean).pow(2)).add(
               bCount.multiply(bMean.subtract(mean).pow(2)))
    
    bss = indices.map(func_bss)
    return means.sort(bss).get([-1])



def Water_Extraction(self):
    """
    Function to extract surface water from satellite images

    args:
        None

    returns:
        None
    """
    try:
        global water_images
        nd_threshold = index_threshold.value
        color_palette = index_color.value       
        # Function to extract water using NDWI or MNDWI from multispectral images
        def extract_water(img):
            """
            Function to extract surface water from Landsat and Sentinel-2 images using
            water extraction indices: NDWI, MNDWI, and AWEI

            args:
                Image

            returns:
                Image with water mask
            """
            index_image = ee.Image(1)
            if water_indices.value == 'NDWI':
                if imageType == 'Landsat':
                    bands = ['green', 'nir']
                elif imageType == 'Sentinel-2':
                    bands = ['green', 'nir']
                elif imageType == 'USDA NAIP':
                    bands = ['G', 'N']
                index_image = img.normalizedDifference(bands).rename('waterMask')
                water_image = index_image.gt(nd_threshold).selfMask().copyProperties(img, ['system:time_start'])
            elif water_indices.value == 'MNDWI':
                if imageType == 'Landsat':
                    bands = ['green', 'swir1']
                elif imageType == 'Sentinel-2':
                    bands = ['green', 'swir1']
                index_image = img.normalizedDifference(bands).rename('waterMask')
                water_image = index_image.gt(nd_threshold).selfMask().copyProperties(img, ['system:time_start'])
            elif water_indices.value == 'AWEInsh':
                if imageType == 'Landsat':
                    index_image = img.expression(
                            '(4 * (GREEN - SWIR1)) - ((0.25 * NIR)+(2.75 * SWIR2))', {
                                'NIR': img.select('nir'),
                                'GREEN': img.select('green'),
                                'SWIR1': img.select('swir1'),
                                'SWIR2': img.select('swir2')
                            }).rename('waterMask')
                elif imageType == 'Sentinel-2':
                    index_image = img.expression(
                            '(4 * (GREEN - SWIR1)) - ((0.25 * NIR)+(2.75 * SWIR2))', {
                                'NIR': img.select('nir'),
                                'GREEN': img.select('green'),
                                'SWIR1': img.select('swir1'),#.resample('bilinear')\
                                #.reproject(crs= img.select('B11').projection().crs().getInfo()["crs"],scale= 10),
                                'SWIR2': img.select('swir2')#.resample('bilinear')#\
                                #.reproject(crs= img.select('B12').projection().crs().getInfo()["crs"],scale= 10)
                            }).rename('waterMask')
                water_image = index_image.gt(nd_threshold).selfMask().copyProperties(img, ['system:time_start'])
            elif water_indices.value == 'AWEIsh':
                if imageType == 'Landsat':
                    index_image = img.expression(
                            '(BLUE + (2.5 * GREEN) - (1.5 * (NIR + SWIR1)) - (0.25 * SWIR2))', {
                                'BLUE':img.select('blue'),
                                'NIR': img.select('nir'),
                                'GREEN': img.select('green'),
                                'SWIR1': img.select('swir1'),
                                'SWIR2': img.select('swir2')
                            }).rename('waterMask')
                elif imageType == 'Sentinel-2':
                    index_image = img.expression(
                            '(BLUE + (2.5 * GREEN) - (1.5 * (NIR + SWIR1)) - (0.25 * SWIR2))', {
                                'BLUE':img.select('blue'),
                                'NIR': img.select('nir'),
                                'GREEN': img.select('green'),
                                'SWIR1': img.select('swir1'),#.resample('bilinear')\
                                #.reproject(crs= img.select('B11').projection().crs().getInfo()["crs"],scale= 10),
                                'SWIR2': img.select('swir2')#.resample('bilinear')#\
                                #.reproject(crs= img.select('B12').projection().crs().getInfo()["crs"],scale= 10)
                            }).rename('waterMask')
                water_image = index_image.gt(nd_threshold).selfMask().copyProperties(img, ['system:time_start'])    
            
            return water_image
        
        
        # Function to extract water from SAR Sentinel 1 images
        def add_S1_waterMask(img):
            """
            Function to extract surface water from Sentinel-1 images Otsu algorithm

            args:
                Image

            returns:
                Image with water mask
            """
            # Compute histogram
            reducers = ee.Reducer.histogram(255,2).combine(reducer2=ee.Reducer.mean(), sharedInputs=True)\
                .combine(reducer2=ee.Reducer.variance(), sharedInputs= True)
            histogram = img.select('VV').reduceRegion(
            reducer=reducers,
            geometry=site.geometry(),
            scale=10,
            bestEffort=True)

            # Calculate threshold via function otsu (see before)
            threshold = otsu(histogram.get('VV_histogram'))
            
            # get watermask
            waterMask = img.select('VV').lt(threshold).rename('waterMask')
            waterMask = waterMask.updateMask(waterMask) #Remove all pixels equal to 0
            return img.addBands(waterMask)
        
        
        if imageType == 'Sentinel-1':
            water_images = clipped_images.map(add_S1_waterMask).select('waterMask')
            visParams = {'min': 0,'max': 1, 'palette': color_palette}
            Map.addLayer(water_images.first(), visParams, 'Water')
        elif imageType == 'Landsat':
            if water_indices.value == 'DSWE':
                DSWE()
            else:
                water_images = clipped_images.map(extract_water)
            Map.addLayer(water_images.first(), {'palette': color_palette}, 'Water')
        else:
            water_images = clipped_images.map(extract_water)
            Map.addLayer(water_images.first(), {'palette': color_palette}, 'Water')

        #ndwi_palette = ['#ece7f2', '#d0d1e6', '#a6bddb', '#74a9cf', '#3690c0', '#0570b0', '#045a8d', '#023858']
        
    except Exception as e:
             print(e)
             print('An error occurred during computation.')

index_threshold.observe(Water_Extraction, 'value')
    
def Sharpen_Image(self):
    """
    Function to pansharpen Landsat and Sentinel-2 images
    Sentinel-2 image pansharpening algorithm yet to be implemented

    args:
        None

    returns:
        None
    """
    try: 
        # function to sharpen Landsat images
        def sharpen_Landsat(img):
            """
            Function to pansharpen Landsat images
            
            args:
                Image

            returns:
                Sharpened image
            """
            hsv = img.select(['B4', 'B3', 'B2']).rgbToHsv()
            sharpened_img = ee.Image.cat([hsv.select('hue'), hsv.select('saturation'), img.select('B8')]).hsvToRgb()
            return sharpened_img
        
        if imageType == 'Landsat':
            visParams = {
              'min': 0,
              'max': 0.25,
              'gamma': [1.1, 1.1, 1.1]}
            sharpened_images = clipped_images.map(sharpen_Landsat)
            Map.addLayer(sharpened_images, visParams, 'L8 Sharpened')
        else:
            # function to sharpen Sentinel-2 images yet to be added
            pass
              
    except Exception as e:
             print(e)
             print('An error occurred during computation.')
    
def maskclouds(self):
    """
    Function to mask clouds from Landsat and Sentinel-2 images
    
    args:
        None

    returns:
        None
    """
    try: 
        
        if imageType == 'Landsat':
            cloudRemovedImages = clipped_images.map(maskLandsatclouds)
            #cloudRemovedImages = clipped_images.map(cloud_mask.landsatSR())
            # Visualization for TOA
            visParams = {'bands': ['B4', 'B3', 'B2'],
                  'min': 0,
                  'max': 0.25,
                  'gamma': [1.1, 1.1, 1]}
            Map.addLayer(cloudRemovedImages, visParams, 'L8 Cloud Masked')
            
        elif imageType == 'Sentinel-2':
            cloudRemovedImages = clipped_images.map(maskS2clouds)
            visParams = {'bands': ['B4', 'B3', 'B2'],
              'min': 0.0,
              'max': 3000}
            Map.addLayer(cloudRemovedImages, visParams, 'S2 Cloud Masked')
        else:
            pass
            
    except Exception as e:
             print(e)
             print('An error occurred during computation.')
                
def calc_area(img):
    """
    Function to calculate area of water pixels

    args:
        Water mask image

    returns:
        Water image with calculated total area of water pixels
    """
    global unit_symbol
    unit = area_unit.value
    divisor = 1
    if unit =='Square Km':
        divisor = 1e6
        unit_symbol = 'Sqkm'
    elif unit =='Hectares':
        divisor = 1e4
        unit_symbol = 'Ha'
    else:
        divisor = 4047
        unit_symbol = 'acre'
        
    pixel_area = img.multiply(ee.Image.pixelArea()).divide(divisor)
    img_area = pixel_area.reduceRegion(**{
        'geometry': site.geometry(),
        'reducer': ee.Reducer.sum(),
        'scale': img_scale,
        'maxPixels': 1e13
    })
    return img.set({'water_area': img_area})

def plot_areas(self):
    """
    Function to plot a time series of calculated water area for each water image
    
    args:
        None

    returns:
        None
    """
    try:
        global df
        # Compute water areas
        water_areas = water_images.map(calc_area)
        water_stats = water_areas.aggregate_array('water_area').getInfo()

        dates = water_images.aggregate_array('system:time_start')\
            .map(lambda d: ee.Date(d).format('YYYY-MM-dd')).getInfo()
        

        with plot_output:
            plot_output.clear_output()
            dates_lst = [datetime.strptime(i, '%Y-%m-%d') for i in dates]
            y = [item.get('waterMask') for item in water_stats]
            df = pd.DataFrame(list(zip(dates_lst,y)), columns=['Date','Area'])

            
            fig = go.Figure([go.Scatter(x=df['Date'], y=df['Area'], name='Water Hydrograph', 
                    mode='lines+markers', line=dict(dash = 'solid', color ='Black', width = 0.5))])

            # plotly figure layout
            fig.update_layout(title = '<b>Surface Water Area Hydrograph<b>', 
                              title_x = 0.5, title_y = 0.90, title_font=dict(family="Arial",size=24),
                              template = "plotly_white",
                              xaxis =dict(title ='<b>Date<b>', linecolor = 'Black'),
                              yaxis=dict(title='Area ('+unit_symbol+')', linecolor = 'Black'),
                              font_family="Arial")
            fig.show()
            
    except Exception as e:
            print(e)
            print('An error occurred during computation.')

def save_data(self):
    file = str(saveFile_name.value)
    filename = file + ".csv"
    df.to_csv(filename, index=False)

def dowload_water_images(self):
        with feedback:
            try:
                path = folder_selector.selected_path
                folder = folder_name.value
                name_Pattern = '{sat}_{system_date}_{imgType}'
                date_pattern = 'ddMMMy'
                extra = dict(sat=imageType, imgType = 'Water')
                if files_to_download.index == 0:
                    download_images = clipped_images
                    extra = dict(sat=imageType, imgType = 'Satellite')
                elif files_to_download.index == 1:
                    download_images = water_images
                    extra = dict(sat=imageType, imgType = 'Water')
                else:
                    download_images = ee.ImageCollection([water_frequency])
                    name_Pattern = '{sat}_{start}_{end}_{imgType}'
                    extra = dict(sat=imageType, imgType = 'Frequency', start=start_date.value.strftime("%x"),
                                 end=end_date.value.strftime("%x"))

                
                if download_location.index == 0:
                    task = geetools.batch.Export.imagecollection.toDrive(
                        collection = download_images,
                        folder = folder,
                        region = site.geometry(),
                        namePattern = name_Pattern,
                        scale = img_scale,
                        datePattern=date_pattern,
                        extra = extra,
                        verbose=True,
                        maxPixels = int(1e13))
                    task
                    #geemap.ee_export_image_collection(clipped_images, out_dir=path, file_per_band=False)
                    #geemap.ee_export_image_collection_to_drive(clipped_images, folder=folder, scale=img_scale)
                else:
                    geemap.ee_export_image_collection(download_images, out_dir=path)

                feedback.clear_output()
                print("Download complete")

            except Exception as e:
                    print(e)
                    print('Download could not be completed')
                    
def water_frequency(self):
    with feedback:
        try:
            global water_frequency
            water_occurence =  water_images.reduce(ee.Reducer.sum())
            water_frequency = water_occurence.divide(water_images.size()).multiply(100)
            visParams = {'min':0, 'max':100, 'palette': ['orange','yellow','blue','darkblue']}
            Map.addLayer(water_frequency, visParams, 'Water Occurence')
        except Exception as e:
                    print(e)
                    print('Frequecy computation could not be completed')
    
def get_dates(col):
    dates = ee.List(col.toList(col.size()).map(lambda img: ee.Image(img).date().format()))
    return dates

def DSWE():
    global dswe_Coll_Clipped
    global water_images
    color_palette = index_color.value
    dem = ee.Image('USGS/NED')
    aoi = site
    cloud_thresh = cloud_threshold.value
    
    def clipImages(img):
            clipped_image = img.clip(aoi).copyProperties(img, ['system:time_start'])
            return clipped_image
        
    
    # Mask clouds, cloud shadows, and snow
    def maskClouds(img):
        qa = img.select(['pixel_qa'])
        clouds = qa.bitwiseAnd(8).neq(0).Or(qa.bitwiseAnd(16).neq(0)).Or(qa.bitwiseAnd(32).neq(0)) # Cloud
        return img.addBands(clouds.rename('clouds')) # Add band of contaminated pixels
    
    # Apply mask
    img_masked = filtered_landsat.map(maskClouds)
    
    # ----------------------------------------------------------------------
    # Calculate hillshade mask
    # ----------------------------------------------------------------------
    def addHillshade(img):
        solar_azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
        solar_zenith = img.get('SOLAR_ZENITH_ANGLE'); # solar altitude = 90-zenith
        solar_altitude = ee.Number(90).subtract(ee.Number(solar_zenith))
        return img.addBands(ee.Terrain.hillshade(dem, solar_azimuth, solar_altitude).rename('hillshade'))

    # Add hillshade bands
    img_hillshade = img_masked.map(addHillshade)
    # ----------------------------------------------------------------------
    # Calculate DSWE indices
    # ----------------------------------------------------------------------
    def addIndices(img):
        # NDVI
        img = img.addBands(img.normalizedDifference(['nir', 'red']).select([0], ['ndvi']))
        # MNDWI (Modified Normalized Difference Wetness Index) = (Green - SWIR1) / (Green + SWIR1)
        img = img.addBands(img.normalizedDifference(['green', 'swir1']).select([0], ['mndwi']))
        # MBSRV (Multi-band Spectral Relationship Visible) = Green + Red
        img = img.addBands(img.select('green').add(img.select('red')).select([0], ['mbsrv'])).toFloat()
        # MBSRN (Multi-band Spectral Relationship Near-Infrared) = NIR + SWIR1
        img = img.addBands(img.select('nir').add(img.select('swir1')).select([0], ['mbsrn']).toFloat())
        # AWEsh (Automated Water Extent Shadow) = Blue + (2.5 * Green) + (-1.5 * mbsrn) + (-0.25 * SWIR2)
        img = img.addBands(img.expression('blue + (2.5 * green) + (-1.5 * mbsrn) + (-0.25 * swir2)', {
             'blue': img.select('blue'),
             'green': img.select('green'),
             'mbsrn': img.select('mbsrn'),
             'swir2': img.select('swir2')
        }).select([0], ['awesh'])).toFloat()
        return img

    # Add indices
    img_indices = img_hillshade.map(addIndices)
    # ----------------------------------------------------------------------
    # ----------------------------------------------------------------------
    # DSWE parameter testing
    # ----------------------------------------------------------------------
    # 1. ========== Function: test MNDWI ===========
    # If (MNDWI > 0.124) set the ones digit (i.e., 00001)
    def test_mndwi(img):
        mask = img.select('mndwi').gt(0.124)
        return img.addBands(mask \
                .bitwiseAnd(0x1F) \
                .rename('mndwi_bit'))

    # 2. ======== Function: compare MBSRV and MBSRN ========
    # If (MBSRV > MBSRN) set the tens digit (i.e., 00010)
    def test_mbsrv_mbsrn(img):
        mask = img.select('mbsrv').gt(img.select('mbsrn'))
        return img.addBands(mask \
                .bitwiseAnd(0x1F) \
                .leftShift(1) \
                .rename('mbsrn_bit'))

    # 3. ======== Function: test AWEsh ========
    # If (AWEsh > 0.0) set the hundreds digit (i.e., 00100)
    def test_awesh(img):
        mask = img.select('awesh').gt(0.0)
        return img.addBands(mask \
                  .bitwiseAnd(0x1F) \
                  .leftShift(2) \
                  .rename('awesh_bit'))

    # 4. ======= Function: test PSW1 ========
    # If (MNDWI > -0.44 && SWIR1 < 900 && NIR < 1500 & NDVI < 0.7) set the thousands digit (i.e., 01000)
    def test_mndwi_swir1_nir(img):
        mask = img.select('mndwi').gt(-0.44) \
                  .And(img.select('swir1').lt(900)) \
                  .And(img.select('nir').lt(1500)) \
                  .And(img.select('ndvi').lt(0.7))
        return img.addBands(mask \
                .bitwiseAnd(0x1F) \
                .leftShift(3) \
                .rename('swir1_bit'))

    # 5. ======= Function: test PSW2 =========
    # If (MNDWI > -0.5 && SWIR1 < 3000 && SWIR2 < 1000 && NIR < 2500 && Blue < 1000) set the ten-thousands digit (i.e., 10000)
    def test_mndwi_swir2_nir(img):
        mask = img.select('mndwi').gt(-0.5) \
                  .And(img.select('swir1').lt(3000)) \
                  .And(img.select('swir2').lt(1000)) \
                  .And(img.select('nir').lt(2500)) \
                  .And(img.select('blue').lt(1000))
        return img.addBands(mask \
                  .bitwiseAnd(0x1F) \
                  .leftShift(4) \
                  .rename('swir2_bit'))

    # Add all bitwise bands to image collection
    img_indices_bit = ee.ImageCollection(img_indices) \
                  .map(test_mndwi) \
                  .map(test_mbsrv_mbsrn) \
                  .map(test_awesh) \
                  .map(test_mndwi_swir1_nir) \
                  .map(test_mndwi_swir2_nir)
    
    # Function: consolidate individual bit bands
    def sum_bit_bands(img):
        bands = img.select(['mndwi_bit', 'mbsrn_bit', 'awesh_bit', 'swir1_bit', 'swir2_bit'])
        summed_bands = bands.reduce(ee.Reducer.bitwiseOr())
        return img.addBands(summed_bands.rename('summed_bit_band'))
    
    # Add individual bit bands to image collection and summarize
    img_indices_bit = ee.ImageCollection(img_indices) \
                  .map(test_mndwi) \
                  .map(test_mbsrv_mbsrn) \
                  .map(test_awesh) \
                  .map(test_mndwi_swir1_nir) \
                  .map(test_mndwi_swir2_nir) \
                  .map(sum_bit_bands)
    # --------------------------------------------------------
    # Produce DSWE layers
    # ----------------------------------------------------------------------
    # Construct slope image from DEM
    #dem = dem.clip(aoi); # removed clipping in an attempt to speed up script
    slope = ee.Terrain.slope(dem)
    # Convert binary code into 4 DSWE categories
    def convert_bin_dswe(img):
        reclass = img.select('summed_bit_band').remap([0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                                20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
                                                30, 31],

                                                [0, 0, 0, 4, 0, 4, 4, 2, 0, 4,
                                                 4, 2, 4, 2, 2, 1, 4, 4, 4, 2,
                                                 4, 2, 2, 1, 3, 2, 2, 1, 2, 1,
                                                 1, 1]).rename('dswe')
        # ID cloud-contaminated pixels
        reclass = reclass.where(img.select('clouds').eq(1), 9)
        # ID shaded areas
        reclass = reclass.where(img.select('hillshade').lte(110), 8)
        # ID slopes
        reclass = reclass.where(img.select('dswe').eq(4) and slope.gte(5.71).Or  # 10% slope = 5.71°
                      (img.select('dswe').eq(3) and slope.gte(11.31)).Or           # 20% slope = 11.31°
                      (img.select('dswe').eq(2) and slope.gte(16.7)).Or            # 30% slope = 16.7°
                      (img.select('dswe').eq(1) and slope.gte(16.7)), 0);          # 30% slope = 16.7°

        return img.addBands(reclass).select('dswe')
    
    img_indices_all = img_indices_bit.map(convert_bin_dswe)
    

    # Viz parameters: classes: 0, 1, 2, 3, 4, 9
    dswe_viz = {'min':0, 'max': 9, 'palette': ['000000', '002ba1', '6287ec', '77b800', 'c1bdb6', '000000', '000000',
                                       '000000', '000000', 'ffffff']}
    
    dswe_Coll_Clipped = img_indices_all.select('dswe').map(clipImages)
    dswe_Coll_Clipped = tools.imagecollection.mosaicSameDay(dswe_Coll_Clipped)
    
    def maskDSWE_Water(img):
        waterImage = img.select('dswe').rename('waterMask')
        watermask = waterImage.gt(0).And(waterImage.lte(4)).selfMask().copyProperties(img, ['system:time_start'])
        return watermask
    
    water_images = dswe_Coll_Clipped.map(maskDSWE_Water)
    water_images = tools.imagecollection.mosaicSameDay(water_images)
    # DSWE image (not composited)
    Map.addLayer(dswe_Coll_Clipped.first(), dswe_viz, "DSWE")
    Map.addLayer(water_images.first(), {'palette': color_palette}, 'Water')
    #Map.addLayer(ee.Image(img_indices_all.select('dswe').first()), dswe_viz, "DSWE")
    # DSWE monthly composite image (from image collection)
    #Map.addLayer(dswe_ic.first().select('dswe'), dswe_viz, "DSWE composite")
    # Landsat Data
    #Map.addLayer(ee.Image(img_indices.first()), ls_viz, 'Landsat')

    
    
imageProcessing_Button.on_click(process_images)
extractWater_Button.on_click(Water_Extraction)
sharpenImage_Button.on_click(Sharpen_Image)
removeCloud_Button.on_click(maskclouds)
plot_button.on_click(plot_areas)
download_button.on_click(dowload_water_images)
water_Frequency_button.on_click(water_frequency)
save_data_button.on_click(save_data)

display(GUI)
display(Map)
display(OUTPUTS)

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

## Download DSWE data only

In [ ]:
name_Pattern = '{sat}_{system_date}_{imgType}'
date_pattern = 'ddMMMy'
extra = dict(sat=imageType, imgType = 'DSWE')

task = geetools.batch.Export.imagecollection.toDrive(
                        collection = dswe_Coll_Clipped,
                        folder = 'DSWE',
                        region = site.geometry(),
                        namePattern = name_Pattern,
                        scale = img_scale,
                        datePattern=date_pattern,
                        extra = extra,
                        verbose=True,
                        crs = 'EPSG:5070',
                        crsTransform = [30, 0, -2214330, 0, -30, 2091360],
                        maxPixels = int(1e13))
task